<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/ViTLarge_Equal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets transformers

In [ ]:
!pip install accelerate -U

In [ ]:
from google.colab import drive
drive.mount('content/')

Drive already mounted at content/; to attempt to forcibly remount, call drive.mount("content/", force_remount=True).


In [ ]:
%cd content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [ ]:
from datasets import load_dataset

dataset = load_dataset("imagefolder", data_dir="Dataset_Equal/")

Resolving data files:   0%|          | 0/1324 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/496 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/452 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1324
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 496
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 452
    })
})

In [ ]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy
import torch
import torchvision
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler

In [ ]:
import shutil

In [ ]:
from transformers import ViTFeatureExtractor

model_name_or_path = 'google/vit-large-patch16-224-in21k'
feature_extractor = ViTFeatureExtractor.from_pretrained(model_name_or_path)

/usr/local/lib/python3.10/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


In [ ]:
def transform(example_batch):
    # Take a list of PIL images and turn them to pixel values
    inputs = feature_extractor([x for x in example_batch['image']], return_tensors='pt')

    # Don't forget to include the labels!
    inputs['labels'] = example_batch['label']
    return inputs

prepared_ds = dataset.with_transform(transform)

In [ ]:
prepared_ds = dataset.with_transform(transform)

In [ ]:
prepared_ds

DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 1324
    })
    validation: Dataset({
        features: ['image', 'label'],
        num_rows: 496
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 452
    })
})

In [ ]:
import numpy as np
from datasets import load_metric

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

<ipython-input-13-223a1e4a98a9>:10: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
from transformers import ViTForImageClassification

labels = prepared_ds['train'].features['label'].names

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)
NUM_CLASSES = 2
BATCH_SIZE = 8
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Some weights of the model checkpoint at google/vit-large-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-large-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
%%capture
!pip install wandb -qU
!pip install weightwatcher

In [ ]:
import wandb
wandb.login()

wandb: Currently logged in as: vedaantj (uiuc-disruption-lab). Use `wandb login --relogin` to force relogin


True

In [ ]:
import weightwatcher as ww

In [ ]:
%env WANDB_PROJECT=funny_not_funny_ViT_Large_Equal

env: WANDB_PROJECT=funny_not_funny_ViT_Large_Equal


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
  output_dir="./vit-large_equal",
  per_device_train_batch_size=8,
  evaluation_strategy="steps",
  num_train_epochs=4,
  fp16=True,
  save_steps=200,
  eval_steps=200,
  logging_steps=10,
  learning_rate=1e-5,
  save_total_limit=2,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='wandb',
  load_best_model_at_end=True,
  optim='adamw_torch',
  weight_decay=0.1
)

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds["train"],
    eval_dataset=prepared_ds["validation"],
    tokenizer=feature_extractor,
)

In [ ]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze()

In [ ]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

Step,Training Loss,Validation Loss,Accuracy
200,0.383000,0.616146,0.665323
400,0.165300,0.640850,0.669355
600,0.071700,0.726127,0.661290


***** train metrics *****
  epoch                    =          4.0
  total_flos               = 1351121269GF
  train_loss               =       0.3436
  train_runtime            =   0:11:55.46
  train_samples_per_second =        7.402
  train_steps_per_second   =        0.928


In [ ]:
watcher = ww.WeightWatcher(model=model)
details = watcher.analyze()
summary = watcher.get_summary(details)
summary

{'log_norm': 3.29,
 'alpha': 5.020873942344486,
 'alpha_weighted': 7.265760438054557,
 'log_alpha_norm': 7.663765552981181,
 'log_spectral_norm': 1.72,
 'stable_rank': 63.4}

In [ ]:
metrics = trainer.evaluate(prepared_ds['validation'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        4.0
  eval_accuracy           =     0.6653
  eval_loss               =     0.6161
  eval_runtime            = 0:00:26.28
  eval_samples_per_second =     18.871
  eval_steps_per_second   =      2.359


In [ ]:
metrics = trainer.predict(prepared_ds['test'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)
wandb.finish()

***** eval metrics *****
  test_accuracy           =     0.6128
  test_loss               =     0.6523
  test_runtime            = 0:02:14.94
  test_samples_per_second =       3.35
  test_steps_per_second   =      0.422


eval/accuracy,▅█▁▅
eval/loss,▁▃█▁
eval/runtime,█▃▄▁
eval/samples_per_second,▁▅▅█
eval/steps_per_second,▁▅▅█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/learning_rate,███▇▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁▁
train/loss,████████▇█▆▅▄▄▄▄▅▄▄▅▃▂▂▃▂▃▂▂▂▂▁▁▁▁▁▁▂▁▁▁
train/total_flos,▁
train/train_loss,▁


In [ ]:
predictions_tensor = torch.tensor(metrics.predictions)

In [ ]:
preds = torch.sigmoid(predictions_tensor)
print(preds.shape)
preds = torch.argmax(preds, dim=1)
preds, preds.shape

torch.Size([452, 2])


(tensor([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
         0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
         1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
         0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0,
         0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1,
         0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0,
         1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1,
         0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
         1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
         0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
         1, 1, 1, 0, 0, 0, 0

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
print(accuracy_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Test Funny', 'Test Not Funny'], index=['Pred Funny', 'Pred Not Funny'])

0.6128318584070797


,Test Funny,Test Not Funny
Pred Funny,134,84
Pred Not Funny,91,143


In [ ]:
len(os.listdir('/content/content/MyDrive/SummerResearch/Dataset/test/Not_Funny_Combined'))

314

In [ ]:
446/539

0.8274582560296846

In [ ]:
314/539

0.5825602968460112

In [ ]:
metrics = trainer.predict(prepared_ds['train'])
trainer.log_metrics("eval", metrics.metrics)
trainer.save_metrics("eval", metrics.metrics)
predictions_tensor = torch.tensor(metrics.predictions)
preds = torch.sigmoid(predictions_tensor)
preds = torch.argmax(preds, dim=1)
print(accuracy_score(preds, metrics.label_ids))
pd.DataFrame(confusion_matrix(preds, metrics.label_ids), columns=['Train Funny', 'Train Not Funny'], index=['Pred Funny', 'Pred Not Funny'])

***** eval metrics *****
  test_accuracy           =     0.9275
  test_loss               =     0.3613
  test_runtime            = 0:01:10.80
  test_samples_per_second =       18.7
  test_steps_per_second   =      2.345
0.9274924471299094


,Train Funny,Train Not Funny
Pred Funny,614,49
Pred Not Funny,47,614
